In [ ]:
a

In [1]:
import os
os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication')
import pandas as pd
import numpy as np
import pickle

In [ ]:
os.chdir(r'H:\MckinseyR\us-grp-nmo-rsm-rba\Alternative_Data_Science\r\Share Code')
for i in os.listdir('.'):
    if i[-2:] == '.R':
        f = open(i).read()
        if 'OfferMedical' in f:
            print(i)

In [ ]:
with open('H:/us-grp-mo-std-rtw/Data_Science/pickles/std_rtw_501_cu_categories_dict.pkl', "rb") as f:
    cu_categories_dict = pickle.load(f)

In [ ]:
import gzip
with gzip.open('H:/us-grp-mo-std-rtw/Data_Science/pickles/std_rtw_501_cu_rf.pkl', "rb") as f:
    cu_smodel = pickle.load(f)

In [ ]:
cu_smodel.n_features_

In [ ]:
with open('H:/us-grp-mo-std-rtw/Data_Science/pickles/std_rtw_501_cu_categories_dict.pkl', "wb") as f:
    pickle.dump(cu_categories_dict, f)

In [ ]:
cu_categories_dict['co_mrbd_surg_txt'] = cu_categories_dict.pop('como_sur')
cu_categories_dict['clm_spcl_tnr_ltd_cvr_txt'] = cu_categories_dict.pop('cstenure_ltdcov')
cu_categories_dict['dif_qtl_1_bin_mdn'] = cu_categories_dict.pop('diff_q1_median_bin')
cu_categories_dict['lkly_clm_prgrs_txt'] = cu_categories_dict.pop('lcp')
cu_categories_dict['mdc_age_intactn_txt'] = cu_categories_dict.pop('mdc_age')
cu_categories_dict['mdc_job_class_1_txt'] = cu_categories_dict.pop('mdc_jobclass1')
cu_categories_dict['mdc_prog_dt_ind'] = cu_categories_dict.pop('mdc_prog')
cu_categories_dict['dpa_tot_task_bin_30_day'] = cu_categories_dict.pop('total_tasks_30days_bin')
cu_categories_dict

In [ ]:
os.chdir(r'\\ns-sisci01\APP_11469_GAIN_Analytics\DataSharing_MetLife - Granularity of Growth\Scaleup Phase\Data\GRIP data\CodeShare\us-grp-nmo-rsm-rba\Alternative_Data_Science\r\Share Code')
for i in os.listdir('.'):
    if i[-2:] == '.R':
        f = open(i).read()
        if 'last_Year_in_int' in f and 'old' not in i and 'not_use' not in i:
            print(i)

In [ ]:
dt = pd.DataFrame()
met_inscope_products = []
met_inscope_state = []
size_cut = []
suffix = ''
metvar = []
met_datawd = ''

In [ ]:
import sys

def met_add_field(dt, RM=True,
                         size_cut=[500,1000,5000],
                         allprods=["Medical", "Dental", "Life", "LTD", "STD", "Vision", "CI", "Accident", "Hospital", "Legal"],
                         mod="met_add_field"):

    beginvar = list(dt.columns)
    beginobs = len(dt)
    
    dt = dt.merge(zip_st_mapping, left_on='Broker_Zip', right_on='Zip', how='left')
    dt['ST'] = dt['ST'].fillna('')
    dt['FL'] = np.where((dt['Broker_State'] != dt['ST']) & (dt['Broker_State'] != '') & (dt['ST'] != ''), 1, 0)
    dt['Broker_State'] = np.where(dt['FL'] == 1, dt['ST'], dt['Broker_State'])
    dt = dt.drop(['ST', 'FL'], axis=1)

    dt = dt.merge(zip_st_mapping, left_on='BrokerCrossProductZIP', right_on='Zip', how='left')
    dt['ST'] = dt['ST'].fillna('')
    dt['FL'] = np.where((dt['BrokerCrossProductSTATE'] != dt['ST']) & (dt['BrokerCrossProductSTATE'] != '') & (dt['ST'] != ''), 1, 0)
    dt['BrokerCrossProductSTATE'] = np.where(dt['FL'] == 1, dt['ST'], dt['BrokerCrossProductSTATE'])
    dt = dt.drop(['ST'], axis=1)

    dt['STATE_EIN'] = dt['STATE_EIN'].astype(str) + dt['EIN'].astype(str)
    dt = dt.sort_values(['STATE_EIN', 'year', 'product', 'Members', 'Carrier'], ascending=[True, True, True, False, True])
    dt['CarrierIND'] = customer.groupby(["STATE_EIN","year","product"])['Carrier'].cumcount(1).add(1)
    map_ = dt[dt['product'] == 'Medical'][['STATE_EIN', 'Carrier', 'CarrierIND', 'year', 'PN', "Members"]]
    map_['CT'] = dt.groupby(["STATE_EIN","CarrierIND","year"])['Members'].count()
    for i in range(1, 3):
        sub = map_[map_['CarrierIND'] == i]
        sub = sub.drop('CarrierIND', axis=1)
        carVar = 'Medical_Car' + str(i)
        sub = sub.rename(columns={'Carrier': carVar})
        dt = dt.merge(sub[['STATE_EIN', 'year', carVar]], on=['STATE_EIN', 'year'], how='left')
        del(sub)
        dt[carVar] = dt[carVar].fillna('')
    dt = dt[dt['product'] != 'Medical']
    dt = dt.merge(met_finance_asmp[['Product', 'PMPM', 'RenewFreq']], left_on='product', right_on='Product', how='inner')
    dt['EstProd_Prem_Carrier'] = 12 * dt['Members'] * dt['PMPM'] / 1000
    dt = add_size_band(dt=dt, size_cut=size_cut)
    if RM:
        dt =dt[dt['Size_Band'] != '5000+']
        for p in allprods:
            t1 = dt[(dt['Carrier'] == 'Metlife') & (dt['product'] == p)]['EstProd_Prem_Carrier'].sum()
            t2 = dt[(dt['Carrier'] == 'Not Provided') & (dt['product'] == p)]['EstProd_Prem_Carrier'].sum()
            ms = t1/t2

    dt = met_add_market(dt, zipvar="Broker_Zip", outputvar="SalesOffice", Statevar="Broker_State",
                      outputvarRegion="Region", brokervar="Broker")
    dt[['SalesOffice', 'Region']] = dt[['SalesOffice', 'Region']].fillna('')
    dt = met_add_market(dt, zipvar="BrokerCrossProductZIP", outputvar="SalesOfficePrimary",
                      outputvarRegion="RegionPrimary", Statevar="BrokerCrossProductSTATE", brokervar="BrokerCrossProduct")
    dt[['SalesOfficePrimary', 'RegionPrimary']] = dt[['SalesOfficePrimary', 'RegionPrimary']].fillna('')
    na = df.isnull().any().any().sum()
    if na > 0:
        print(mod, 'has NAs')
        sys.exit()
    byvars = ["year","STATE_EIN","PN","Carrier","product","Funding"]
    dt['CT'] = dt.groupby(byvars)['Members'].count()
    if dt[dt['CT'] > 1]['CT'].sum() > 0:
        print('not unique at ', byvars)
        sys.exit()
    return dt

In [ ]:
d = {'col1': [2, 2], 'col2': [3, 3]}
df = pd.DataFrame(data=d)
df['col3'] = df.groupby('col1')['col2'].cumcount(1).add(1)
df

In [ ]:
outputdir = 'D:/'
export_file_name = 'abc'
output_path = os.path.join(outputdir, export_file_name + '_' + str(datetime.now().strftime('%Y_%m_%d-%H_%M_%S')) + '.txt')
output_path

In [ ]:
from datetime import datetime

In [ ]:
df.isnull().any().any().sum()

In [ ]:
def check_numeric(x):
    try:
        x = int(x)
        return ''
    except:
        return 'Provided field value is an incorrect data type. ICD_CD_VER is expecting a NUMERIC value to be provided.'

In [ ]:
d = {'col1': [2, 2], 'col2': [3, 3]}
df = pd.DataFrame(data=d)
df['col3'] = df.groupby('col1')['col2'].cumcount(1).add(1)
df['col4'] = ''
df = df.append({'col4': '0'}, ignore_index=True)
df

In [ ]:
df.loc[:, 'x'] = df['col4'].apply(check_numeric)
df

In [ ]:
df['col5'] = df.groupby('col1')['col2'].sum().iloc[0]

In [ ]:
df

In [ ]:
df.groupby('col1')['col2'].sum().iloc[0]

In [ ]:
try:
    int('a')
except ValueError:
    print('a')

In [ ]:
df = pd.DataFrame({'col1': [10, 20, 30, 40], 'col2': ['a', 'a', 'b', 'c']})
value = [40, 20, 10, 30]
value = list(map(lambda x: str(x), value))
#df['col1'].astype(str).astype("category", categories=value)

In [ ]:
df = df.groupby('col2').agg({'col1': [('sss', 'sum'), ('mmm', 'mean')]}).reset_index()

In [ ]:
df.columns = [i[0] if i[1] == '' else i[1] for i in df.columns]

In [ ]:
df.groupby('col2')['col1'].cumcount()+1

In [ ]:
list(range(2014, 2018))

In [ ]:
a = """if x['ICD10to9_2'] != 'Derive_3':
        return x['ICD10to9_2']
    elif x['PRI_ICD_CD'] == 'S52021M':
        return '73382'
    else:
        return 'Unknown'"""

In [ ]:
ca = [i.split(':')[0].strip() for i in a.split('\n')]
cb = ['np.where(' + i.replace('elif ', '').replace('if ', '') if ('if ' in i or 'elif ' in i) else i for i in ca]
cb = [i.replace('else', '') if 'else' in i else i for i in cb]
cb_c = cb[::2]
cb_r = [i.replace('return ', '') for i in cb[1::2]]

ss = 'return '
for i, j in zip(cb_c, cb_r):
    if i != '':
        ss+= i + ', ' + j + ',\n'
    else:
        ss+= j + ','
ss = ss[:-1] + ')'
print(ss)

In [ ]:
ca = [i.split(':')[0].strip() for i in a.split('\n')]
cb = ['(' + i.replace('elif ', '').replace('if ', '') + ')' if ('if ' in i or 'elif ' in i) else i for i in ca]
cb = [i.replace('else', '') if 'else' in i else i for i in cb]
cb_c = cb[::2]
cb_r = [i.replace('return ', '') for i in cb[1::2]]

ss = 'return '
for i, j in zip(cb_c, cb_r):
    if i != '':
        ss+= i + ', ' + j + ',\n'
    else:
        ss+= j + ','
ss = ss[:-1] + ')' * (len(cb_c)-1)

In [ ]:
conditions = cb_c[:-1]
choices = cb_r[:-1]
conditions = [i.replace('x[', 'df[') for i in conditions]
print(conditions)

In [ ]:
#df['new_col'] = np.select(conditions, choices, default=cb_r[-1])
cb_r[-1]

In [ ]:
[exec(i) for i in conditions]

In [3]:
import os
os.chdir('H:\opti')
import pandas as pd
import numpy as np

In [32]:
spss = pd.read_csv('output_27Nov.txt', delimiter='|')
spss = spss.drop(['RequestReceivedTS', 'RequestReturnedTS'], axis=1)
spss = spss.fillna('Was NULL')
spss['ApprovalRate'] = round(spss['ApprovalRate'], 5)

In [33]:
pyop = pd.read_csv('std_dsaa_results_set_2019_11_27-08_09_32.txt', delimiter='|')
pyop = pyop.drop(['RequestReceivedTS', 'RequestReturnedTS'], axis=1)
pyop = pyop.fillna('Was NULL')
pyop['ApprovalRate'] = round(pyop['ApprovalRate'], 5)

In [34]:
(pyop == spss).all()

TransId                     True
ClaimNumberCode             True
AutoAdjudicationDecision    True
ApprovalRate                True
HaveSurgeryDate             True
HavePrognosisDate           True
HaveHospitalizationDate     True
XeroxGroup                  True
ClusterType                 True
Microsegment                True
ResultReasonDesc1           True
ResultReasonDesc2           True
ResultReasonDesc3           True
ResultReasonDesc4           True
ResultReasonDesc5           True
dtype: bool

In [16]:
pyop.shape, spss.shape

((19791, 15), (19791, 15))

In [26]:
pyop[pyop['HaveSurgeryDate'] != spss['HaveSurgeryDate']]

TransId  ClaimNumberCode AutoAdjudicationDecision  \
153    5c4733bb7c7b282333ea6b0d     201901228190                        U   
182    5c48e94d7c7b282333ea6df8     201901238930                        U   
209    5c4b8c607c7b282333ea718a     201901258710                        U   
329    5c5a26397c7b282333ea7bfd     201902058520                        U   
607    5c76bff6a505aa8fc1b9d2cb     201902278930                        U   
702    5c814578a505aa8fc1b9de94     201903078320                        U   
751    5c869852a505aa8fc1b9e35a     201903118220                        U   
911    5c93f54c6ec0df232151c77a     201903218610                        U   
928    5c9524d06ec0df232151c8cc     201903228840                        U   
1075   5c33bcbb7c7b282333ea60c7     211901078531                        U   
1129   5c3e010a7c7b282333ea66ad     211901158571                        U   
1139   5c3f69337c7b282333ea679b     211901168031                        U   
1179   5c4768fa7c7b282333ea6bae     211901228381                        U   
1456   5c658bc27c7b282333ea8629     211902148151                        U   
1514   5c6c7b85a505aa8fc1b9c7c6     211902198641                        U   
1567   5c705f8da505aa8fc1b9cdc6     211902228741                        U   
1582   5c7456c1a505aa8fc1b9cfe1     211902258251                        U   
1696   5c7ec191a505aa8fc1b9da73     211903058581                        U   
1882   5c8ff03e6ec0df232151bf45     211903188351                        U   
2069   5c2fcedf7c7b282333ea5fa4     221901048082                        U   
2196   5c461e7a7c7b282333ea6a98     221901218042                        U   
2212   5c47459f7c7b282333ea6b48     221901228252                        U   
2296   5c507fa07c7b282333ea73ab     221901298322                        U   
2304   5c50aefc7c7b282333ea743e     221901298492                        U   
2315   5c52002e7c7b282333ea7587     221901308832                        U   
2345   5c5487ba7c7b282333ea77e8     221902018462                        U   
2417   5c5c57b97c7b282333ea7dc6     221902078242                        U   
2467   5c62c9c57c7b282333ea8236     221902128312                        U   
2624   5c702944a505aa8fc1b9cd24     221902228602                        U   
2641   5c743e09a505aa8fc1b9cf73     221902258132                        U   
...                         ...              ...                      ...   
16162  5c8c02bba505aa8fc1b9eb98     301903158250                        U   
16173  5c929a986ec0df232151c589     301903208730                        U   
16204  5c2d20d37c7b282333ea5dcc     311901028502                        U   
16416  5c59c7467c7b282333ea7b39     311902058632                        U   
16438  5c5c65937c7b282333ea7df1     311902078972                        U   
16465  5c644dc57c7b282333ea8439     311902138641                        U   
16497  5c6d5ff8a505aa8fc1b9c816     311902208401                        U   
16608  5c818a63a505aa8fc1b9df57     311903078292                        U   
16613  5c82ad69a505aa8fc1b9e14e     311903088372                        U   
16762  5c38d34a7c7b282333ea6507     321901118773                        U   
17082  5c87b48ca505aa8fc1b9e4ab     321903128714                        U   
17092  5c8986fea505aa8fc1b9e86b     321903138984                        U   
17268  5c462e4d7c7b282333ea6ab5     331901218816                        U   
17293  5c49c9bd7c7b282333ea6f16     331901248256                        U   
17393  5c50dcb77c7b282333ea74b4     331901298876                        U   
17577  5c797d13a505aa8fc1b9d68a     331903018595                        U   
17656  5c8f8e026ec0df232151bdeb     331903188285                        U   
17746  5c3629e47c7b282333ea6287     341901098417                        U   
17879  5c51fc7d7c7b282333ea7578     341901308987                        U   
18030  5c7709f7a505aa8fc1b9d3dd     341902278348                        U   
18105  5c88fdf0a505a

In [27]:
spss[spss['HaveSurgeryDate'] != pyop['HaveSurgeryDate']]

TransId  ClaimNumberCode AutoAdjudicationDecision  \
153    5c4733bb7c7b282333ea6b0d     201901228190                        U   
182    5c48e94d7c7b282333ea6df8     201901238930                        U   
209    5c4b8c607c7b282333ea718a     201901258710                        U   
329    5c5a26397c7b282333ea7bfd     201902058520                        U   
607    5c76bff6a505aa8fc1b9d2cb     201902278930                        U   
702    5c814578a505aa8fc1b9de94     201903078320                        U   
751    5c869852a505aa8fc1b9e35a     201903118220                        U   
911    5c93f54c6ec0df232151c77a     201903218610                        U   
928    5c9524d06ec0df232151c8cc     201903228840                        U   
1075   5c33bcbb7c7b282333ea60c7     211901078531                        U   
1129   5c3e010a7c7b282333ea66ad     211901158571                        U   
1139   5c3f69337c7b282333ea679b     211901168031                        U   
1179   5c4768fa7c7b282333ea6bae     211901228381                        U   
1456   5c658bc27c7b282333ea8629     211902148151                        U   
1514   5c6c7b85a505aa8fc1b9c7c6     211902198641                        U   
1567   5c705f8da505aa8fc1b9cdc6     211902228741                        U   
1582   5c7456c1a505aa8fc1b9cfe1     211902258251                        U   
1696   5c7ec191a505aa8fc1b9da73     211903058581                        U   
1882   5c8ff03e6ec0df232151bf45     211903188351                        U   
2069   5c2fcedf7c7b282333ea5fa4     221901048082                        U   
2196   5c461e7a7c7b282333ea6a98     221901218042                        U   
2212   5c47459f7c7b282333ea6b48     221901228252                        U   
2296   5c507fa07c7b282333ea73ab     221901298322                        U   
2304   5c50aefc7c7b282333ea743e     221901298492                        U   
2315   5c52002e7c7b282333ea7587     221901308832                        U   
2345   5c5487ba7c7b282333ea77e8     221902018462                        U   
2417   5c5c57b97c7b282333ea7dc6     221902078242                        U   
2467   5c62c9c57c7b282333ea8236     221902128312                        U   
2624   5c702944a505aa8fc1b9cd24     221902228602                        U   
2641   5c743e09a505aa8fc1b9cf73     221902258132                        U   
...                         ...              ...                      ...   
16162  5c8c02bba505aa8fc1b9eb98     301903158250                        U   
16173  5c929a986ec0df232151c589     301903208730                        U   
16204  5c2d20d37c7b282333ea5dcc     311901028502                        U   
16416  5c59c7467c7b282333ea7b39     311902058632                        U   
16438  5c5c65937c7b282333ea7df1     311902078972                        U   
16465  5c644dc57c7b282333ea8439     311902138641                        U   
16497  5c6d5ff8a505aa8fc1b9c816     311902208401                        U   
16608  5c818a63a505aa8fc1b9df57     311903078292                        U   
16613  5c82ad69a505aa8fc1b9e14e     311903088372                        U   
16762  5c38d34a7c7b282333ea6507     321901118773                        U   
17082  5c87b48ca505aa8fc1b9e4ab     321903128714                        U   
17092  5c8986fea505aa8fc1b9e86b     321903138984                        U   
17268  5c462e4d7c7b282333ea6ab5     331901218816                        U   
17293  5c49c9bd7c7b282333ea6f16     331901248256                        U   
17393  5c50dcb77c7b282333ea74b4     331901298876                        U   
17577  5c797d13a505aa8fc1b9d68a     331903018595                        U   
17656  5c8f8e026ec0df232151bdeb     331903188285                        U   
17746  5c3629e47c7b282333ea6287     341901098417                        U   
17879  5c51fc7d7c7b282333ea7578     341901308987                        U   
18030  5c7709f7a505aa8fc1b9d3dd     341902278348                        U   
18105  5c88fdf0a505a

In [28]:
conditions = "[(df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] == 'M2390') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'ENDOCRINE SYSTEM') & (df['PRI_ICD_CD'] =='27801') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NEOPLASMS') & (df['PRI_ICD_CD'] == 'D259') & (df['V1']== 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F1020') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='O82') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'ENDOCRINE SYSTEM') & (df['PRI_ICD_CD'] =='27801') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'ENDOCRINE SYSTEM') & (df['PRI_ICD_CD'] =='E6601') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='O82') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F332') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M1711') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='6697') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M1711') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='55090') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='71516') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M1712') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8360') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NEOPLASMS') & (df['PRI_ICD_CD'] == '2189') & (df['V1']== 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='O82') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NEOPLASMS') & (df['PRI_ICD_CD'] == 'D259') & (df['V1']== 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'UNKNOWN') & (df['PRI_ICD_CD'] == '74') & (df['V1']== 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='71516') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='K4090') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='6697') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='6697') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='717') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7363') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'ENDOCRINE SYSTEM') & (df['PRI_ICD_CD'] =='E6601') & (df['V1'] == 'Yes') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7366') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '29633') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='4660') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='550') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F322') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='3540') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'Yes'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='486') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72210') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='J209') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='J189') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '29623') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7271') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NEOPLASMS') & (df['PRI_ICD_CD'] == '2189') & (df['V1']== 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='575') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'GENITOURINARY SYSTEM') & (df['PRI_ICD_CD'] =='N949') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'CIRCULATORY SYSTEM') & (df['PRI_ICD_CD'] =='I519') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='311') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F331') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M5126') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72210') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72761') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '29632') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M542') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8360') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='J189') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='O82') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '2962') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7242') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M5126') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M545') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='K829') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='6697') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '29633') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8404') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F1020') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='30002') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'DIGESTIVE  SYSTEM') & (df['PRI_ICD_CD'] =='575') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F320') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7244') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='3540') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7231') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F332') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72210') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7261') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='G5601') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'GENITOURINARY SYSTEM') & (df['PRI_ICD_CD'] =='6218') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='G5600') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72252') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'GENITOURINARY SYSTEM') & (df['PRI_ICD_CD'] =='N949') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='72210') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7245') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F329') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F411') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7366') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8360') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F430') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='717') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='30000') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F419') & (df['V1'] == 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7244') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='9999') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F331') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M5126') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='O80') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'NORMAL PREGNANCY) & (DELIVERY') & (df['PRI_ICD_CD'] =='650') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'Yes'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='71946') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='J189') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M542') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7242') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7231') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='30002') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'COMPLICATIONS OF PREGNANCY') & (df['PRI_ICD_CD'] =='646') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'UNKNOWN') & (df['PRI_ICD_CD'] == 'Z759') & (df['V1']== 'No') & (df['V2'] == 'Yes') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F329') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8470') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F411') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M545') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F320') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='3540') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='486') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='8472') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='30981') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='M549') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'CIRCULATORY SYSTEM') & (df['PRI_ICD_CD'] =='I10') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='311') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == '2962') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'UNKNOWN') & (df['PRI_ICD_CD'] == 'Z759') & (df['V1']== 'Yes') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F419') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='30000') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='S3992XA') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'RESPIRATORY SYSTEM') & (df['PRI_ICD_CD'] =='487') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'PSYCHOSES') & (df['PRI_ICD_CD'] == 'F328') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='7245') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'MUSCULOSKELETAL SYSTEM') & (df['PRI_ICD_CD'] =='724') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NERVOUS SYSTEM') & (df['PRI_ICD_CD'] =='346') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F430') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='9597') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'UNKNOWN') & (df['PRI_ICD_CD'] == 'Z759') & (df['V1']== 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='3089') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='9591') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'NON-PSYCHOTIC MENTAL DISORDERS') & (df['PRI_ICD_CD'] =='F6810') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='T1490') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='999') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'INJURY) & (POISONING') & (df['PRI_ICD_CD'] =='9599') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'ILL DEFINED CONDITIONS') & (df['PRI_ICD_CD'] =='7809') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'ILL DEFINED CONDITIONS') & (df['PRI_ICD_CD'] =='780') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No'),   (df['MDC'] == 'AIDS) & (ARC') & (df['PRI_ICD_CD'] == '42') & (df['V1'] == 'No') & (df['V2'] == 'No') & (df['V3'] == 'No')]"
choices = ['6', '8', '10', '14', '16', '19', '20', '25', '31', '38', '42', '44', '45', '49', '53', '54', '56', '64', '67', '74', '75', '79', '80', '86', '89', '90', '96', '98', '99', '101', '102', '103', '105', '112', '113', '114', '128', '129', '130', '136', '137', '138', '140', '144', '145', '153', '166', '169', '170', '184', '188', '189', '192', '194', '205', '206', '209', '213', '214', '217', '220', '228', '233', '237', '238', '239', '240', '245', '246', '247', '249', '253', '257', '258', '262', '263', '264', '267', '272', '273', '274', '276', '277', '281', '285', '286', '291', '293', '297', '299', '301', '304', '305', '306', '307', '309', '319', '320', '322', '327', '329', '337', '339', '344', '353', '354', '356', '358', '360', '367', '371', '375', '378', '381', '383', '384', '385', '390', '392', '393', '394', '395', '396', '397', '398', '400', '401', '402', '403', '404', '405', '407', '408', '410', '411', '412', '414', '415', '416', '417', '418', '419', '420', '180']

In [27]:
choices.index('247')

69

In [30]:
len(choices)

144

In [69]:
f = open('H:/opti/std_dlcp_results_set.txt').read()
print(f.split('\n')[0])

TransId	ClaimNumberCode	EDFlag	RTWinthepast	ClusterType	EDSuspicousLevel	Microsegment	Co_morbidity	PrognosisDateLessThan30	LCPPredicted	ResultReasonDesc1	ResultReasonDesc2	ResultReasonDesc3	ResultReasonDesc4	ResultReasonDesc5	RequestReceivedTS	RequestReturnedTS


In [73]:
df = pd.read_csv('H:/opti/std_dlcp_results_set.txt', delimiter='\t')#, parse_dates=['DABL_DT', 'RTRN_TO_WRK_DT'])
df = df.drop(['ClusterType', 'EDSuspicousLevel'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13741 entries, 0 to 13740
Data columns (total 15 columns):
TransId                    13741 non-null object
ClaimNumberCode            13741 non-null int64
EDFlag                     142 non-null object
RTWinthepast               13596 non-null float64
Microsegment               13596 non-null float64
Co_morbidity               13741 non-null int64
PrognosisDateLessThan30    13596 non-null float64
LCPPredicted               13741 non-null int64
ResultReasonDesc1          13741 non-null object
ResultReasonDesc2          13596 non-null object
ResultReasonDesc3          13596 non-null float64
ResultReasonDesc4          13596 non-null float64
ResultReasonDesc5          13596 non-null float64
RequestReceivedTS          13741 non-null object
RequestReturnedTS          13741 non-null object
dtypes: float64(6), int64(3), object(6)
memory usage: 1.6+ MB


In [75]:
df.dropna().head(1)

TransId  ClaimNumberCode EDFlag  RTWinthepast  \
97  5c2d06017c7b282333ea5da6     301901028430   Flag           0.0   

    Microsegment  Co_morbidity  PrognosisDateLessThan30  LCPPredicted  \
97        1184.0             0                      0.0             2   

   ResultReasonDesc1   ResultReasonDesc2  ResultReasonDesc3  \
97          ICD=3320  MDC=NERVOUS SYSTEM                0.0   

    ResultReasonDesc4  ResultReasonDesc5    RequestReceivedTS  \
97                0.0                0.0  2019-10-11 03:55:16   

      RequestReturnedTS  
97  2019-10-11 03:55:22

In [55]:
df.head()

TRANS_ID    CLM_NUM_CD    DABL_DT RTRN_TO_WRK_DT  \
0  5c8aa1ada505aa8fc1b9e99d  251903148505 2019-04-02     2019-04-17   
1  5c8c03b4a505aa8fc1b9eb9c  201903158950 2019-03-19     2019-04-30   
2  5c990e646ec0df232151cabc  501903258428 2019-03-10     2019-04-15   
3  5c9a3faf6ec0df232151cc80  281903268608 2019-03-15     2019-06-15   
4  5c6efd8fa505aa8fc1b9cb1a  261902218306 2019-04-08     2019-06-03   

   CLM_RECD_DT  CVR_CD  
0          NaN     NaN  
1          NaN     NaN  
2          NaN     NaN  
3          NaN     NaN  
4          NaN     NaN